In [1]:
import pandas as pd
import numpy as np
import warnings
import pyreadstat
import torch
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from collections import Counter

if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = 'cpu'

print(f"Device: '{device}'")
warnings.filterwarnings('ignore')

Device: 'mps'


# 데이터 불러오기

In [2]:
import os 

base_path = '../data/HN/'
folders = ['24RC', 'ALL']
years = range(2020, 2024)

diet_raw_data = pd.DataFrame()
all_raw_data = pd.DataFrame()

diet_meta_dict = {}
all_meta_dict = {}


for year in years:
    year_str = str(year)
    
    for folder in folders:
        file_path = os.path.join(base_path, folder, f'HN{year_str[-2:]}.sav')
        
        if folder == '24RC':
            raw_data, meta_data = pyreadstat.read_sav(file_path)

            raw_data['ID'] = year_str + raw_data['ID']

            meta_df = pd.DataFrame([meta_data.column_names, meta_data.column_labels]).transpose()
            diet_meta_dict[year_str] = meta_df

            diet_raw_data = pd.concat([diet_raw_data, raw_data], axis=0)
        
        elif folder == 'ALL':
            raw_data, meta_data = pyreadstat.read_sav(file_path)

            raw_data['ID'] = year_str + raw_data['ID']

            meta_df = pd.DataFrame([meta_data.column_names, meta_data.column_labels]).transpose()
            all_meta_dict[year_str] = meta_df

            all_raw_data = pd.concat([all_raw_data, raw_data], axis=0)

In [6]:
def calculate_activity_levels(df):
    def get_minutes_per_week_vec(yn_col, days_col, hours_col, minutes_col):
        yn_mask = df[yn_col] == 1
        result = np.zeros(len(df))
        result[yn_mask] = df.loc[yn_mask, days_col] * (df.loc[yn_mask, hours_col] * 60 + df.loc[yn_mask, minutes_col])
        return result
    
    high_work = get_minutes_per_week_vec('BE3_71', 'BE3_72', 'BE3_73', 'BE3_74')
    high_leisure = get_minutes_per_week_vec('BE3_75', 'BE3_76', 'BE3_77', 'BE3_78')
    moderate_work = get_minutes_per_week_vec('BE3_81', 'BE3_82', 'BE3_83', 'BE3_84')
    moderate_leisure = get_minutes_per_week_vec('BE3_85', 'BE3_86', 'BE3_87', 'BE3_88')
    transport = get_minutes_per_week_vec('BE3_91', 'BE3_92', 'BE3_93', 'BE3_94')
    
    total_minutes = (high_work + high_leisure) * 2 + moderate_work + moderate_leisure + transport
    
    activity_levels = np.ones(len(df), dtype=int)  # 기본값: 1 (비활동적)
    activity_levels[(total_minutes >= 150) & (total_minutes < 300)] = 2  # 저활동적
    activity_levels[(total_minutes >= 300) & (total_minutes < 600)] = 3  # 활동적
    activity_levels[total_minutes >= 600] = 4  # 매우 활동적
    
    return activity_levels

def normalization(data):
    numeric_cols = []
    categorical_cols = []
    
    # 컬럼 유형 분류
    for col in data.columns:
        non_null_values = data[col].dropna()
        unique_count = len(non_null_values.unique())
        
        is_numeric = True
        try:
            pd.to_numeric(non_null_values)
        except:
            is_numeric = False
        
        if unique_count <= 15 or col.startswith(('sex', 'incm', 'edu', 'occp', 'DI', 'DF', 'BD', 'BE')):
            categorical_cols.append(col)
        elif is_numeric:
            numeric_cols.append(col)

    # 수치형 컬럼 스케일링
    if numeric_cols:
        scaled_features = StandardScaler().fit_transform(data[numeric_cols].fillna(0))
        for i, col in enumerate(numeric_cols):
            data[col] = scaled_features[:, i]

    # 범주형 컬럼 원핫인코딩 
    cols_to_drop = []  # 원래 변수를 삭제하기 위한 리스트
    
    for col in categorical_cols:
        unique_vals = sorted(data[col].dropna().unique())
        encoded_cols_created = False  # 인코딩 컬럼 생성 여부 확인
        
        for val in unique_vals:
            if not pd.isna(val):
                try:
                    col_name = f"{col}_{int(val) if isinstance(val, (int, float)) else val}"
                    data[col_name] = (data[col] == val).astype(float)
                    encoded_cols_created = True
                except:
                    col_name = f"{col}_{val}"
                    data[col_name] = (data[col] == val).astype(float)
                    encoded_cols_created = True
        
        # 원핫인코딩 컬럼이 생성되었으면 원래 변수를 삭제 리스트에 추가
        if encoded_cols_created:
            cols_to_drop.append(col)
    
    # 원래 변수 삭제
    data = data.drop(columns=cols_to_drop)
    
    return data

## User data

In [4]:
# 1. 관심 있는 사용자 특성 컬럼 확장
list23 = all_meta_dict['2023'][0].tolist()
list22 = all_meta_dict['2022'][0].tolist()
list21 = all_meta_dict['2021'][0].tolist()
list20 = all_meta_dict['2020'][0].tolist()

user_cols = list(set(list20) & set(list21) & set(list22) & set(list23))
extended_user_data = all_raw_data[user_cols].copy()

# 2. 활동량 계산
activity_cols = ['BE3_71', 'BE3_72', 'BE3_73', 'BE3_74', 'BE3_81', 'BE3_82', 'BE3_83', 'BE3_84', 
                'BE3_91', 'BE3_92', 'BE3_93', 'BE3_94', 'BE3_75', 'BE3_76', 'BE3_77', 'BE3_78',
                'BE3_85', 'BE3_86', 'BE3_87', 'BE3_88']

if all(col in extended_user_data.columns for col in activity_cols):
    extended_user_data['activity_level'] = calculate_activity_levels(extended_user_data)
else:
    missing_cols = [col for col in activity_cols if col not in extended_user_data.columns]
    extended_user_data['activity_level'] = 2

# 3. 정규화
extended_user_data = extended_user_data.set_index('ID')
for col in extended_user_data.columns:
    if extended_user_data[col].apply(lambda x: isinstance(x, str)).any():
        extended_user_data = extended_user_data.drop(col, axis=1)
extended_user_data = extended_user_data.fillna(0)
user_features = normalization(extended_user_data).dropna(axis=1)

## Intake data

In [ ]:
food_cols = ['N_DCODE', 'N_DNAME', 'N_MEAL', 'N_FCODE', 'N_FNAME', 'N_KINDG1', 'N_FM_WT']
nutri_cols = ['NF_EN', 'NF_CHO', 'NF_PROT', 'NF_FAT', 'NF_SFA', 'NF_CHOL', 'NF_TDF', 'NF_SUGAR', 'NF_NA', 'NF_CA', 'NF_PHOS', 'NF_K']

user_food_data = diet_raw_data[['ID', 'N_TD_VOL', 'N_TD_WT'] + food_cols + nutri_cols].copy()

# 섭취 부피, 중량 처리
user_food_data['N_TD'] = np.where(user_food_data['N_TD_VOL'].notna(), user_food_data['N_TD_VOL'], user_food_data['N_TD_WT'])
user_food_data = user_food_data.drop(['N_TD_VOL', 'N_TD_WT'], axis=1)

# 1회 제공량 처리
N_CD_data = pd.read_csv('../data/HN/24RC/N_CD.csv')

user_food_data['음식코드'] = user_food_data['N_DCODE'].astype(str).str[-4:].astype(int)
user_food_data = user_food_data.merge(N_CD_data, on='음식코드', how='left')
user_food_data = user_food_data.drop('음식코드', axis=1)

# 소비량 계산
user_food_data['consumed_ratio'] = user_food_data['N_TD'] / user_food_data['N_CD']

# 음식 코드 처리
name_to_code = {name: f"{i:02d}" for i, name in enumerate(user_food_data['N_DNAME'].unique())}
user_food_data['O_DCODE'] = user_food_data['N_DCODE']
user_food_data['N_DCODE'] = user_food_data['O_DCODE'].astype(str) + user_food_data['N_DNAME'].map(name_to_code)
user_food_data = user_food_data.drop_duplicates()

# 식품 카테고리 특성 추출
user_food_data['major_group'] = user_food_data['O_DCODE'].str[1:3].astype(int)
user_food_data['sub_group'] = user_food_data['O_DCODE'].str[3:5].astype(int)

# 선호도 스코어 계산
user_food_data['label'] = user_food_data['consumed_ratio'] > 0.5

intake_data = user_food_data[['ID', 'N_DCODE', 'consumed_ratio','label']].copy().drop_duplicates()

## Diet data

In [10]:
# 식품 데이터 추출
food_data = user_food_data[food_cols + nutri_cols + ['N_TD', 'major_group', 'sub_group']].copy()

# 결측치 처리
food_data = food_data.fillna(0)
food_data['N_KINDG1'] = food_data['N_KINDG1'].astype(int)

# 영양소 특성 정규화
scaler = StandardScaler()
food_data[nutri_cols] = scaler.fit_transform(food_data[nutri_cols])

# 그룹별 영양소 특성 계산
food_agg = food_data.groupby('N_DCODE')[nutri_cols].mean()
category_agg = food_data.groupby(['major_group', 'sub_group'])[nutri_cols].mean()

# 최종 식품 데이터
unique_foods = food_data[['N_DCODE', 'N_DNAME', 'major_group', 'sub_group']].drop_duplicates('N_DCODE')
food_features = unique_foods.merge(food_agg, on='N_DCODE')

# 데이터 분석

In [11]:
print("\n=== 데이터 분석 요약 ===")
print(f"사용자 수: {user_features.index.nunique()}")
print(f"식품 수: {food_features['N_DCODE'].nunique()}")
print(f"상호작용 수: {len(intake_data)}")
print(f"양성 상호작용 비율: {intake_data['label'].mean():.2f}")


=== 데이터 분석 요약 ===
사용자 수: 27643
식품 수: 35310
상호작용 수: 323308
양성 상호작용 비율: 0.56


## 클러스터링

### 데이터 준비

In [18]:
from sklearn.cluster import KMeans
from scipy.sparse import csr_matrix
import umap

cluster_data = user_food_data[['ID','N_DCODE','consumed_ratio']].dropna(subset=['consumed_ratio'])
cluster_data = cluster_data.groupby(['ID', 'N_DCODE'])['consumed_ratio'].mean().reset_index()

user_ids = cluster_data['ID'].unique()
food_ids = cluster_data['N_DCODE'].unique()

user_to_idx = {id: i for i, id in enumerate(user_ids)}
food_to_idx = {id: i for i, id in enumerate(food_ids)}

# 인덱스로 변환
user_idx = cluster_data['ID'].map(user_to_idx).values
food_idx = cluster_data['N_DCODE'].map(food_to_idx).values
values = cluster_data['consumed_ratio'].values

# 희소 행렬 생성
interaction_matrix = csr_matrix((values, (user_idx, food_idx)), shape=(len(user_ids), len(food_ids)))

# 차원 축소
reducer = umap.UMAP(n_components=100, random_state=42)
reduced_features = reducer.fit_transform(interaction_matrix.toarray())

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


### 최적 클러스터 수

In [ ]:
from sklearn.metrics import silhouette_score

# Elbow method와 실루엣 점수로 최적 k 찾기
distortions = []
silhouette_scores = []
K_range = range(2, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(reduced_features)
    distortions.append(kmeans.inertia_)
    if k > 1:  # 실루엣 점수는 클러스터가 2개 이상일 때만 계산 가능
        silhouette_scores.append(silhouette_score(reduced_features, kmeans.labels_))
    else:
        silhouette_scores.append(0)

# 시각화
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(K_range, distortions, 'bx-')
plt.xlabel('Cluster N (k)')
plt.ylabel('Distortion')
plt.title('Elbow Method')

plt.subplot(1, 2, 2)
plt.plot(K_range, silhouette_scores, 'bx-')
plt.xlabel('Cluster N (k)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Method')
plt.tight_layout()
plt.show()

### 클러스터링 및 특성 분석

In [20]:
# 시각화 결과를 바탕으로 최적의 k 선택
optimal_k = 6  # 위 그래프 결과에 따라 조정

# K-means 클러스터링 수행
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
intake_clusters = kmeans.fit_predict(reduced_features)

In [ ]:
# PCA로 차원 축소 및 시각화
pca = PCA(n_components=2)
pca_result = pca.fit_transform(reduced_features)

# PCA 결과 시각화
plt.figure(figsize=(10, 8))
scatter = plt.scatter(pca_result[:, 0], pca_result[:, 1], c=intake_clusters, alpha=0.6, cmap='viridis')
plt.colorbar(scatter, label='Cluster')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('PCA Visualization of Clusters')
plt.show()

### 사용자 특성과 통계 분석

In [74]:
# 사용자 특성 데이터 준비
# 기존 user_features에서 식품 섭취 관련 특성을 제외한 사용자 특성만 선택
user_demographic_features = user_features.copy()
for col in user_demographic_features.columns:
    if any(nutrient in col for nutrient in ['N_']):
        user_demographic_features = user_demographic_features.drop(col, axis=1)

cluster_result = pd.DataFrame({'ID': user_ids, 'cluster': intake_clusters}).set_index('ID')

# 사용자 특성과 클러스터 결합
user_with_clusters = cluster_result.join(user_demographic_features)

# 범주형 특성과 클러스터 간의 관계 (카이제곱 검정)
from scipy.stats import chi2_contingency
def cramers_v(confusion_matrix):
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2/n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2-((k-1)*(r-1))/(n-1))
    rcorr = r-((r-1)**2)/(n-1)
    kcorr = k-((k-1)**2)/(n-1)
    return np.sqrt(phi2corr/min((kcorr-1), (rcorr-1)))

categorical_features = []
for col in user_with_clusters.columns:
    if col != 'cluster' and user_with_clusters[col].nunique() < 15:
        categorical_features.append(col)
cat_significance = {}
cat_strength = {}
for feature in categorical_features:
    contingency_table = pd.crosstab(user_with_clusters['cluster'], user_with_clusters[feature])
    if contingency_table.shape[0] > 1 and contingency_table.shape[1] > 1:
        chi2, p, dof, expected = chi2_contingency(contingency_table)
        v = cramers_v(contingency_table)
        cat_significance[feature] = p
        cat_strength[feature] = v

# 관계 강도 기준 정렬 (유의미한 것들만)
significant_categorical = [(f, v) for f, v in cat_strength.items() if cat_significance[f] < 0.05]
significant_categorical = sorted(significant_categorical, key=lambda x: x[1], reverse=True)[:30]

# 연속형 특성과 클러스터 간의 관계 (ANOVA)
from scipy.stats import f_oneway
def eta_squared(df, feature, cluster_col='cluster'):
    classes = df[cluster_col].unique()
    grand_mean = df[feature].mean()
    between_ss = sum([len(df[df[cluster_col] == c]) * (df[df[cluster_col] == c][feature].mean() - grand_mean)**2 for c in classes])
    
    total_ss = sum((df[feature] - grand_mean)**2)
    return between_ss / total_ss if total_ss > 0 else 0

numerical_features = []
for col in user_with_clusters.columns:
    if col != 'cluster' and user_with_clusters[col].nunique() >= 15:
        numerical_features.append(col)

num_significance = {}
num_strength = {}
for feature in numerical_features:
    groups = [user_with_clusters[user_with_clusters['cluster'] == i][feature].dropna() for i in range(optimal_k)]
    groups = [g for g in groups if len(g) > 0]
    if len(groups) > 1:  # 최소 2개 그룹 필요
        f_stat, p = f_oneway(*groups)
        eta_sq = eta_squared(user_with_clusters.dropna(subset=[feature]), feature)
        num_significance[feature] = p
        num_strength[feature] = eta_sq

# 관계 강도 기준 정렬 (유의미한 것들만)
significant_numerical = [(f, v) for f, v in num_strength.items() if num_significance[f] < 0.05]
significant_numerical = sorted(significant_numerical, key=lambda x: x[1], reverse=True)[:30]

In [75]:
feature_list = pd.concat([all_meta_dict['2020'], all_meta_dict['2021'], all_meta_dict['2022'], all_meta_dict['2023']])
feature_list.columns = ['Feature', 'Label']
feature_list.drop_duplicates(subset=['Feature'], inplace=True)

top_num_features_df = pd.DataFrame(significant_numerical)
top_num_features_df.columns = ['Feature','Strength']
top_num_features_df = top_num_features_df.merge(feature_list, on='Feature', how='left')

top_cat_features_df = pd.DataFrame(significant_categorical)
top_cat_features_df.columns = ['Feature_cat','Strength']
top_cat_features_df['Feature'] = top_cat_features_df['Feature_cat'].str.rsplit('_', n=1).str[0]
top_cat_features_df = top_cat_features_df.merge(feature_list, on='Feature', how='left')

top_stats_df = pd.concat([top_cat_features_df, top_num_features_df])

In [76]:
top_stats_df

,Feature_cat,Strength,Feature,Label
0,dr_month_0,0.281629,dr_month,월간음주율
1,dr_month_1,0.281629,dr_month,월간음주율
2,BD1_11_6,0.229217,BD1_11,(만12세이상) 1년간 음주빈도
3,BD2_1_8,0.226948,BD2_1,(만12세이상) 한번에 마시는 음주량
4,BD7_4_3,0.221663,BD7_4,(성인) 가족/의사의 절주 권고 여부
5,BD1_11_5,0.217297,BD1_11,(만12세이상) 1년간 음주빈도
6,BE3_75_8,0.198873,BE3_75,고강도 신체활동 여부: 여가
7,BS12_32_8,0.198873,BS12_32,(성인) 평생사용담배종류: 물담배
8,BS12_33_8,0.198873,BS12_33,(성인) 평생사용담배종류: 시가
9,BE3_91_8,0.198873,BE3_91,신체활동 여부: 장소이동
